In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib widget

In [ ]:
from lsst.utils.plotting import publication_plots
from lsst.utils.plotting import get_multiband_plot_colors

colors = get_multiband_plot_colors()
bands = colors.keys()

In [ ]:
from lsst.summit.utils import (
    getAirmassSeeingCorrection,
    getBandpassSeeingCorrection,
)

In [ ]:
CAM_FWHM = 0.207 # arcsec

In [ ]:
import os
os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcam'

In [ ]:
def fetch(day_obs_min, day_obs_max):
    
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    ccdvisit1_quicklook.psf_ixx,
    ccdvisit1_quicklook.psf_ixy,
    ccdvisit1_quicklook.psf_iyy,
    ccdvisit1.detector,
    visit1.visit_id,
    visit1.seq_num,
    visit1.band,
    visit1.physical_filter,
    visit1.day_obs,
    visit1.target_name,
    visit1.science_program,
    visit1.observation_reason,
    visit1.airmass,
    visit1.altitude,
    visit1.azimuth,
    visit1.sky_rotation,
    visit1.exp_midpt_mjd,
    visit1.dimm_seeing,
    visit1.s_ra,
    visit1.s_dec,
    visit1_quicklook.psf_sigma_min,
    visit1_quicklook.psf_sigma_median,
    visit1_quicklook.aos_fwhm,
    visit1_quicklook.donut_blur_fwhm,
    visit1_quicklook.ringss_seeing,
    visit1_quicklook.seeing_zenith_500nm_min,
    visit1_quicklook.seeing_zenith_500nm_median,
    visit1_quicklook.physical_rotator_angle
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1,
    cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161) -- vignetted
    AND ccdvisit1.detector NOT IN (191, 192, 195, 196, 199, 200, 203, 204) -- corner wavefront sensors
    AND visit1.airmass > 0
    AND visit1.day_obs >= {day_obs_min} AND visit1.day_obs <= {day_obs_max}
    AND visit1.science_program in ('BLOCK-365', 'BLOCK-407', 'BLOCK-408', 'BLOCK-416', 'BLOCK-417', 'BLOCK-419');
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
    ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")
    ccdvisits["donut_blur_fwhm"] = pd.to_numeric(ccdvisits["donut_blur_fwhm"], errors="coerce")
    ccdvisits["aos_fwhm"] = pd.to_numeric(ccdvisits["aos_fwhm"], errors="coerce")

    ccdvisits["psf_e1"] = (ccdvisits["psf_ixx"] - ccdvisits["psf_iyy"]) / (ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"])
    ccdvisits["psf_e2"] = (2. * ccdvisits["psf_ixy"]) / (ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"])
    ccdvisits["psf_e"] = np.sqrt(np.array(ccdvisits["psf_e1"]**2 + ccdvisits["psf_e2"]**2, dtype=float))

    ccdvisits["psf_e"] = pd.to_numeric(ccdvisits["psf_e"], errors="coerce")
    ccdvisits["psf_e1"] = pd.to_numeric(ccdvisits["psf_e1"], errors="coerce")
    ccdvisits["psf_e2"] = pd.to_numeric(ccdvisits["psf_e2"], errors="coerce")

    return ccdvisits

In [ ]:
day_obs_min = 20250119
day_obs_max = 20260119
ccdvisits = fetch(day_obs_min, day_obs_max)

In [ ]:
airmass_seeing_correction = np.array([getAirmassSeeingCorrection(airmass) for airmass in ccdvisits["airmass"]])
bandpass_seeing_correction = np.array([getBandpassSeeingCorrection(band) for band in ccdvisits["physical_filter"]])
ccdvisits["psf_fwhm_zenith_500nm"] = ccdvisits["psf_fwhm"] * airmass_seeing_correction * bandpass_seeing_correction

In [ ]:
ccdvisits['donut_blur_atm_fwhm'] = np.sqrt(ccdvisits['donut_blur_fwhm']**2 - CAM_FWHM**2)
ccdvisits['aos_cam_fwhm'] = np.sqrt(ccdvisits['aos_fwhm']**2 + CAM_FWHM**2)

In [ ]:
groups = ccdvisits.groupby('visit_id')
visits_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'target_name': groups['target_name'].first(),
    'science_program': groups['science_program'].first(),
    'observation_reason': groups['observation_reason'].first(),
    'seq_num': groups['seq_num'].median(),
    'exp_midpt_mjd': groups['exp_midpt_mjd'].median(),
    'donut_blur_fwhm': groups['donut_blur_fwhm'].median(),
    'aos_fwhm': groups['aos_fwhm'].median(),
    'donut_blur_atm_fwhm': groups['donut_blur_atm_fwhm'].median(),
    'aos_cam_fwhm': groups['aos_cam_fwhm'].median(),
    'physical_rotator_angle': groups['physical_rotator_angle'].median(),
    'altitude': groups['altitude'].median(),
    'psf_fwhm_05': groups['psf_fwhm'].quantile(0.05),
    'psf_fwhm_50': groups['psf_fwhm'].quantile(0.50),
    'psf_fwhm_95': groups['psf_fwhm'].quantile(0.95),
    'psf_fwhm_zenith_500nm_50': groups['psf_fwhm_zenith_500nm'].quantile(0.50),
    'psf_e_05': groups['psf_e'].quantile(0.05),
    'psf_e_50': groups['psf_e'].quantile(0.50),
    'psf_e_95': groups['psf_e'].quantile(0.95),
    'psf_e1_05': groups['psf_e1'].quantile(0.05),
    'psf_e1_50': groups['psf_e1'].quantile(0.50),
    'psf_e1_95': groups['psf_e1'].quantile(0.95),
    'psf_e2_05': groups['psf_e2'].quantile(0.05),
    'psf_e2_50': groups['psf_e2'].quantile(0.50),
    'psf_e2_95': groups['psf_e2'].quantile(0.95),
    'band': groups['band'].first(),
})
visits_summary['psf_fwhm_95_05'] = np.sqrt(visits_summary['psf_fwhm_95']**2 - visits_summary['psf_fwhm_05']**2)
visits_summary['sys_50'] = np.sqrt(visits_summary['psf_fwhm_50']**2 + CAM_FWHM**2 - visits_summary['donut_blur_fwhm']**2)
visits_summary['psf_fwhm_model'] = np.sqrt(visits_summary['aos_fwhm']**2 + visits_summary['donut_blur_fwhm']**2)

In [ ]:
# Exclude y due to issues with estimating donut blur
selection = (visits_summary['band'] != "y") & ~visits_summary['observation_reason'].str.contains('filter_change_close_loop')

groups = visits_summary[selection].groupby('day_obs')
day_obs_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'n_visits': groups['day_obs'].count(),
    'psf_fwhm_95_05_low': groups['psf_fwhm_95_05'].quantile(0.10),
    'psf_fwhm_95_05_50': groups['psf_fwhm_95_05'].quantile(0.50),
    'psf_fwhm_95_05_high': groups['psf_fwhm_95_05'].quantile(0.90),
    'aos_fwhm_low': groups['aos_fwhm'].quantile(0.10),
    'aos_fwhm_50': groups['aos_fwhm'].quantile(0.50),
    'aos_fwhm_high': groups['aos_fwhm'].quantile(0.90),
    'aos_cam_fwhm_low': groups['aos_cam_fwhm'].quantile(0.10),
    'aos_cam_fwhm_50': groups['aos_cam_fwhm'].quantile(0.50),
    'aos_cam_fwhm_high': groups['aos_cam_fwhm'].quantile(0.90),
    'sys_50_low': groups['sys_50'].quantile(0.10),
    'sys_50_50': groups['sys_50'].quantile(0.50),
    'sys_50_high': groups['sys_50'].quantile(0.90),
    'psf_e_50_low': groups['psf_e_50'].quantile(0.10),
    'psf_e_50_50': groups['psf_e_50'].quantile(0.50),
    'psf_e_50_high': groups['psf_e_50'].quantile(0.90),
    'psf_e1_50_low': groups['psf_e1_50'].quantile(0.10),
    'psf_e1_50_50': groups['psf_e1_50'].quantile(0.50),
    'psf_e1_50_high': groups['psf_e1_50'].quantile(0.90),
    'psf_e2_50_low': groups['psf_e2_50'].quantile(0.10),
    'psf_e2_50_50': groups['psf_e2_50'].quantile(0.50),
    'psf_e2_50_high': groups['psf_e2_50'].quantile(0.90),
    'psf_fwhm_50_low': groups['psf_fwhm_50'].quantile(0.10),
    'psf_fwhm_50_50': groups['psf_fwhm_50'].quantile(0.50),
    'psf_fwhm_50_high': groups['psf_fwhm_50'].quantile(0.90),
    'donut_blur_atm_fwhm_low': groups['donut_blur_atm_fwhm'].quantile(0.10),
    'donut_blur_atm_fwhm_50': groups['donut_blur_atm_fwhm'].quantile(0.50),
    'donut_blur_atm_fwhm_high': groups['donut_blur_atm_fwhm'].quantile(0.90),
})